<a href="https://colab.research.google.com/github/euphoria96/KB_Smishing_dacon/blob/master/preprocess1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
cd /content/gdrive/My Drive/Colab Notebooks/smishing/Mecab-ko-for-Google-Colab

/content/gdrive/My Drive/Colab Notebooks/smishing/Mecab-ko-for-Google-Colab


In [0]:
! bash install_mecab-ko_on_colab190912.sh

Installing konlpy.....
     |████████████████████████████████| 19.4MB 230kB/s 
     |████████████████████████████████| 491kB 42.9MB/s 
     |████████████████████████████████| 92kB 12.8MB/s 
  Created wheel for JPype1: filename=JPype1-0.7.1-cp36-cp36m-linux_x86_64.whl size=2452173 sha256=493952d2c4a88fa048bf62f89d722dcdd0961a0d2b3b9223e626afd3a8e44b41
  Stored in directory: /root/.cache/pip/wheels/b0/49/6a/4393ef8542c00becf80691bd242693db9e263d6e499323a984
Successfully built JPype1
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2020-01-04 05:46:33--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket

In [0]:
# data preprocessing
import pandas as pd
import numpy as np
import random
# tokenizer
import re
from konlpy.tag import Mecab
import warnings
warnings.filterwarnings(action='ignore')

In [0]:
cd /content/gdrive/My Drive/Colab Notebooks/smishing/

/content/gdrive/My Drive/Colab Notebooks/smishing


In [0]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/public_test.csv')
submission=pd.read_csv('data/submission_제출양식.csv')
print(train.shape, test.shape, submission.shape)
train.head(3)

(295945, 4) (1626, 3) (1626, 2)


,id,year_month,text,smishing
0,0,2017-01,XXX은행성산XXX팀장입니다.행복한주말되세요,0
1,1,2017-01,오늘도많이웃으시는하루시작하세요XXX은행 진월동VIP라운지 XXX올림,0
2,2,2017-01,안녕하십니까 고객님. XXX은행입니다.금일 납부하셔야 할 금액은 153600원 입니...,0


In [0]:
def text_preprocessing(text_list):
    stopwords = ['을', '를', '이', '가', '은', '는', 'null']
    tokenizer = Mecab()
    token_list, ngram2_list, ngram3_list = [], [], []
    for text in text_list:
        txt = re.sub('[^가-힣a-z]',' ',text.lower())
        txt = re.sub('x{1,}',' ',txt)
        token = tokenizer.morphs(txt)
        token = [t for t in token if t not in stopwords or type(t)!= float]
        token_list.append(' '.join(token))
        ngram2 = [token[i]+'.'+token[i+1] for i in range(len(token)-1)]
        ngram3 = [token[i]+'.'+token[i+1]+'.'+token[i+2] for i in range(len(token)-2)]
        ngram2_list.append(' '.join(ngram2))
        ngram3_list.append(' '.join(ngram3))
    return token_list, ngram2_list, ngram3_list

In [0]:
train['text_'], train['2gram'], train['3gram'] = text_preprocessing(train.text)
test['text_'], test['2gram'], test['3gram'] = text_preprocessing(test.text)

In [0]:
train.head(2)

,id,year_month,text,smishing,text_,2gram,3gram
0,0,2017-01,XXX은행성산XXX팀장입니다.행복한주말되세요,0,은행 성산 팀장 입니다 행복 한 주말 되 세요,은행.성산 성산.팀장 팀장.입니다 입니다.행복 행복.한 한.주말 주말.되 되.세요,은행.성산.팀장 성산.팀장.입니다 팀장.입니다.행복 입니다.행복.한 행복.한.주말 ...
1,1,2017-01,오늘도많이웃으시는하루시작하세요XXX은행 진월동VIP라운지 XXX올림,0,오늘 도 많이 웃 으시 는 하루 시작 하 세요 은행 진월동 vip 라운지 올림,오늘.도 도.많이 많이.웃 웃.으시 으시.는 는.하루 하루.시작 시작.하 하.세요 ...,오늘.도.많이 도.많이.웃 많이.웃.으시 웃.으시.는 으시.는.하루 는.하루.시작 ...


In [0]:
train.iloc[95753]

id                111663
year_month       2017-06
text          XXX6ㅛㅗ8ㅛㅗ8
smishing               0
text_                   
2gram                   
3gram                   
Name: 95753, dtype: object

In [0]:
for i,t in enumerate(train['text_']):
    if t == '':
        print(i,t)
        train['text_'][] = ' '
        train['2gram'] = ' '
        train['3gram'] = ' '

95753 


In [0]:
test.iloc[486]

id             340486
year_month    2019-02
text             4444
text_                
2gram                
3gram                
Name: 486, dtype: object

In [0]:
train.to_csv('data/train_processed.csv', index=False)
test.to_csv('data/test_processed.csv', index=False)